In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import logging
import datetime

from sklearn.preprocessing import OneHotEncoder,MinMaxScaler, Normalizer, LabelEncoder
from sklearn.feature_selection import SelectKBest,chi2,SelectFromModel
from xgboost import XGBClassifier, XGBRegressor
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from xgboost import plot_importance
from mlxtend.preprocessing import DenseTransformer
from mlxtend.feature_selection import ColumnSelector
from itertools import product

sns.set(color_codes=True)

In [2]:
def _transfer_type(df, cols, dtype):
    for v in cols:
        df[v] = df[v].astype(dtype)
    
    return df

# Features Engineering

## outliers

In [3]:
dateparse = lambda x: pd.datetime.strptime(x, '%d.%m.%Y')
sales_df = pd.read_csv('../input/sales_train.csv', parse_dates = ['date'], date_parser=dateparse)

item_df = pd.read_csv('../input/items.csv')
shop_df = pd.read_csv('../input/shops.csv')
category_df = pd.read_csv('../input/item_categories.csv')

test_df = pd.read_csv('../input/test.csv').set_index('ID')

In [4]:
sales_df = _transfer_type(sales_df, ['date_block_num', 'shop_id', 'item_id', 'item_cnt_day'], np.int16)
sales_df = _transfer_type(sales_df, ['item_price'], np.float16)

item_df = _transfer_type(item_df, ['item_id', 'item_category_id'], np.int16)
item_df['item_name'] = item_df['item_name'].astype(str)

shop_df['shop_name'] = shop_df['shop_name'].astype(str)
shop_df['shop_id'] = shop_df['shop_id'].astype(np.int16)

category_df['item_category_name'] = category_df['item_category_name'].astype(str)
category_df['item_category_id'] = category_df['item_category_id'].astype(np.int16)

### Clear the data out of range

In [5]:
sales_df = sales_df[sales_df.item_price<100000]
sales_df = sales_df[sales_df.item_cnt_day<1000]

### Correct the negative item price

In [6]:
sales_df[sales_df.item_price < 0]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
484683,2013-05-15,4,32,2973,-1.0,1


In [7]:
sales_df.at[484683, 'item_price'] = sales_df[(sales_df.item_id == 2973) & (sales_df.item_price > 0)].item_price.mean()

### Adjust duplicates by shop name

According to the reference, some shops are duplicates. This feature found is based on the language background.

In [8]:
shop_df[shop_df.shop_id == 0]

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0


In [9]:
shop_df[shop_df.shop_id == 57]

,shop_name,shop_id
57,"Якутск Орджоникидзе, 56",57


The shop_id 0 and 57 are the same shop. And other pairs are 1 and 58, 10 and 11.

In [10]:
# Якутск Орджоникидзе, 56
sales_df.at[sales_df.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
sales_df.at[sales_df.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
sales_df.at[sales_df.shop_id == 10, 'shop_id'] = 11

## Extract Features

In [11]:
def _rename(prefix):
    cols = ['2013-01', '2013-02', '2013-03', '2013-04','2013-05','2013-06','2013-07','2013-08','2013-09','2013-10','2013-11','2013-12','2014-01','2014-02','2014-03','2014-04','2014-05','2014-06','2014-07','2014-08','2014-09','2014-10','2014-11','2014-12','2015-01','2015-02','2015-03','2015-04','2015-05','2015-06','2015-07','2015-08', '2015-09','2015-10']
    
    result = {}
    for i in range(1, len(cols) + 1, 1):
        result[cols[i-1]] = prefix + str(i)
    
    return result

+ **Feature 'r' series**: Sales record

In [12]:
import gc
t = sales_df.groupby([sales_df.date.apply(lambda x: x.strftime('%Y-%m')),'item_id','shop_id'])['item_cnt_day'].sum().reset_index(name='item_cnt_month')

t = t[['date','item_id','shop_id','item_cnt_month']]
t = t.pivot_table(index=['item_id','shop_id'], columns='date',values='item_cnt_month',fill_value=0).reset_index()
t = t.rename(index=str, columns=_rename('r'))
sales_detail_df = t.copy()

del t
gc.collect()

35

In [13]:
# leak_df = test_df[['item_id', 'shop_id']].drop_duplicates()
# sales_detail_df = pd.merge(sales_detail_df, leak_df, on=['item_id', 'shop_id'], how='outer')
# sales_detail_df = sales_detail_df.fillna(0)
# sales_detail_df.info()

In [14]:
def _extract(df, i, is_test=False):
    arr = ['item_id','shop_id']
    for j in range(1, 13, 1):
        arr = arr + ['r' + str(i-j)]
    
    if not is_test:
        arr = arr + ['r' + str(i)]
    
    tdf = df[arr]
    tdf['date_block_num'] = i - 1
    
    return tdf

In [15]:
cols = ['item_id','shop_id', 'r1', 'r2', 'r3', 'r4',  'r5', 'r6','r7', 'r8',  'r9', 'r10',  'r11',  'r12', 'item_cnt_month', 'date_block_num']
sales_record_df = pd.DataFrame(columns=cols)
for i in range(13, 35, 1):
    tdf = _extract(sales_detail_df, i)
    sales_record_df = sales_record_df.append(pd.DataFrame(columns=cols, data=tdf.values))

sales_record_df['month'] = sales_record_df['date_block_num'].apply(lambda x: (x % 12) + 1)

for i in range(1, 13, 1):
    sales_record_df['r'+str(i)] = sales_record_df['r'+str(i)].astype(np.float16)

sales_record_df = _transfer_type(sales_record_df, ['item_id', 'shop_id', 'date_block_num', 'month'], np.int16)
sales_record_df = _transfer_type(sales_record_df, ['item_cnt_month'], np.float16)
sales_record_df = sales_record_df.fillna(0)

/Users/leewind/.local/share/virtualenvs/leewind-p6XO93Th/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


+ **Feature 'city'**: Each shop_name starts with the city name.

In [16]:
shop_df.loc[shop_df.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
shop_df['shop_name'] = shop_df['shop_name'].astype(str)
shop_df['city'] = shop_df['shop_name'].str.split(' ').map(lambda x: x[0])
shop_df.loc[shop_df.city == '!Якутск', 'city'] = 'Якутск'

encoder = LabelEncoder()
encoder.fit(shop_df['city'])
shop_df['city_code'] = encoder.transform(shop_df['city'])

+ **Feature 'type'**: Each category contains type and subtype in its name.
+ **Feature 'subtype'**: Each category contains type and subtype in its name.

In [17]:
category_df['item_category_name'] = category_df['item_category_name'].astype(str)
category_df['type'] = category_df['item_category_name'].map(lambda x: x.split('-')[0].strip())

encoder = LabelEncoder()
encoder.fit(category_df['type'])
category_df['type_code'] = encoder.transform(category_df['type'])

In [18]:
category_df['subtype'] = category_df['item_category_name'].map(lambda x: x.split('-')[1].strip() if len(x.split('-')) > 1 else x.split('-')[0].strip())

encoder = LabelEncoder()
encoder.fit(category_df['subtype'])
category_df['subtype_code'] = encoder.transform(category_df['subtype'])

## Merge Data into one entity

In [19]:
sales_record_df = pd.merge(sales_record_df, item_df, on=['item_id'], how='left')
sales_record_df = pd.merge(sales_record_df, shop_df, on=['shop_id'], how='left')
sales_record_df = pd.merge(sales_record_df, category_df, on=['item_category_id'], how='left')
sales_record_df = sales_record_df.fillna(0)

sales_record_df = sales_record_df.drop(['city', 'type', 'subtype', 'item_name', 'shop_name', 'item_category_name'], axis=1)
sales_record_df = _transfer_type(sales_record_df, ['item_category_id', 'city_code', 'type_code', 'subtype_code'], np.int16)

sales_record_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9215976 entries, 0 to 9215975
Data columns (total 21 columns):
item_id             int16
shop_id             int16
r1                  float16
r2                  float16
r3                  float16
r4                  float16
r5                  float16
r6                  float16
r7                  float16
r8                  float16
r9                  float16
r10                 float16
r11                 float16
r12                 float16
item_cnt_month      float16
date_block_num      int16
month               int16
item_category_id    int16
city_code           int16
type_code           int16
subtype_code        int16
dtypes: float16(13), int16(8)
memory usage: 439.5 MB


In [20]:
def _agg(df, cols, prefix):
    
    result = df[cols].drop_duplicates()
    
    for i in range(1, 13, 1):
        t = df.groupby(cols)['r'+str(i)].mean().fillna(0).astype(np.float16).reset_index(name=prefix+str(i))
        result = pd.merge(result, t, on=cols, how='left')
    
#     result = _mean(result, prefix)
    return result

In [21]:
sc_df = _agg(sales_record_df, ['shop_id', 'item_category_id'], 'sc')
i_df = _agg(sales_record_df, ['item_id'], 'i')
it_df = _agg(sales_record_df, ['item_category_id'], 'it')
s_df = _agg(sales_record_df, ['shop_id'], 's')

sales_record_df = pd.merge(sales_record_df, sc_df, on=['shop_id', 'item_category_id'], how='left')
sales_record_df = pd.merge(sales_record_df, i_df, on=['item_id'], how='left')
sales_record_df = pd.merge(sales_record_df, it_df, on=['item_category_id'], how='left')
sales_record_df = pd.merge(sales_record_df, s_df, on=['shop_id'], how='left')

## Feature Selection

In [22]:
dataset = sales_record_df.copy()
dataset.columns.values

array(['item_id', 'shop_id', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7',
       'r8', 'r9', 'r10', 'r11', 'r12', 'item_cnt_month',
       'date_block_num', 'month', 'item_category_id', 'city_code',
       'type_code', 'subtype_code', 'sc1', 'sc2', 'sc3', 'sc4', 'sc5',
       'sc6', 'sc7', 'sc8', 'sc9', 'sc10', 'sc11', 'sc12', 'i1', 'i2',
       'i3', 'i4', 'i5', 'i6', 'i7', 'i8', 'i9', 'i10', 'i11', 'i12',
       'it1', 'it2', 'it3', 'it4', 'it5', 'it6', 'it7', 'it8', 'it9',
       'it10', 'it11', 'it12', 's1', 's2', 's3', 's4', 's5', 's6', 's7',
       's8', 's9', 's10', 's11', 's12'], dtype=object)

In [23]:
dataset_beta = dataset[dataset.date_block_num < 33]
dataset_alpha = dataset[dataset.date_block_num == 33]

In [24]:
features = [
    'item_id', 'shop_id', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7',
    'r8', 'r9', 'r10', 'r11', 'r12',
    'date_block_num', 'month', 'item_category_id', 'city_code',
    'type_code', 'subtype_code',
    
    'sc1', 'sc2', 'sc3', 'sc4', 'sc5', 'sc6', 'sc7', 'sc8', 'sc9', 'sc10', 'sc11', 'sc12',
    'i1', 'i2', 'i3', 'i4', 'i5', 'i6', 'i7', 'i8', 'i9', 'i10', 'i11', 'i12',
    'it1', 'it2', 'it3', 'it4', 'it5', 'it6', 'it7', 'it8', 'it9', 'it10', 'it11', 'it12',
    's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12',
]
label = 'item_cnt_month'

## Train

In [25]:
import xgboost as xgb

dataset_beta = dataset[(dataset.date_block_num < 24) & (dataset.date_block_num > 17)]
train_dataset_x = dataset_beta[features]
train_dataset_y = dataset_beta[label].values.ravel()
train_dmatrix = xgb.DMatrix(train_dataset_x, label=train_dataset_y)

dataset_test = dataset[(dataset.date_block_num == 33)]
test_dataset_x = dataset_test[features]
test_dataset_y = dataset_test[label].values.ravel()
test_dmatrix = xgb.DMatrix(test_dataset_x, label=test_dataset_y)

dataset_valid = dataset[(dataset.date_block_num == 32)]
valid_dataset_x = dataset_valid[features]
valid_dataset_y = dataset_valid[label].values.ravel()
valid_dmatrix = xgb.DMatrix(valid_dataset_x, label=valid_dataset_y)

In [26]:
from xgboost import XGBRegressor

watchlist = [(train_dmatrix, 'train'), (valid_dmatrix, 'validate')] 

params = {
  'booster': 'gbtree',
  'objective': 'reg:linear',
  'eta': 0.05,
  'gamma': 0.7,
  'min_child_weight': 7,
  'max_depth': 4,
  'subsample': 0.5,
  'colsample_bytree': 0.1,
  'nthread': 4,
  'silent': 0,
  'seed': 2019,
  "max_evals": 200,
}

watchlist = [(train_dmatrix, 'train'), (valid_dmatrix, 'validate')] 
bst = xgb.train(params, train_dmatrix, evals=watchlist, early_stopping_rounds=10, num_boost_round=1000)

[13:45:25] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[0]	train-rmse:3.34076	validate-rmse:4.95164
Multiple eval metrics have been passed: 'validate-rmse' will be used for early stopping.

Will train until validate-rmse hasn't improved in 10 rounds.
[13:45:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=4
[1]	train-rmse:3.28133	validate-rmse:4.93103
[13:45:30] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[2]	train-rmse:3.24683	validate-rmse:4.91746
[13:45:32] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=4
[3]	train-rmse:3.18941	validate-rmse:4.89514
[13:45:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=4
[4]	train-rmse:3.1255	validate-rmse:4.87443
[13:45:36] src/tree/updater_prune.cc:74: tree pruning end, 1 root

In [27]:
model = xgb.train(params, train_dmatrix, num_boost_round=bst.best_iteration)

[13:47:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[13:47:10] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=4
[13:47:12] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[13:47:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=4
[13:47:16] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=4
[13:47:17] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=4
[13:47:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=4
[13:47:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=4
[13:47:23] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_

In [28]:
from sklearn.metrics import mean_squared_error
from math import sqrt

test_dmatrix = xgb.DMatrix(dataset_test[features], label=dataset_test[label].values.ravel())
pred = model.predict(test_dmatrix)
sqrt(mean_squared_error(dataset_test[label], pred))

/Users/leewind/.local/share/virtualenvs/leewind-p6XO93Th/lib/python3.7/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


1.5550207806391543

## Predict

In [67]:
tdf = _extract(sales_detail_df, 35, is_test=True)
cols = ['item_id','shop_id', 'r1', 'r2', 'r3', 'r4',  'r5', 'r6','r7', 'r8',  'r9', 'r10',  'r11',  'r12', 'date_block_num']
dataset_predict_df = pd.DataFrame(columns=cols, data=tdf.values)
dataset_predict_df = dataset_predict_df.drop_duplicates(['item_id','shop_id'])

dataset_predict_df = pd.merge(test_df, dataset_predict_df, on=['item_id', 'shop_id'], how='left')
dataset_predict_df['date_block_num'] = 34
dataset_predict_df = dataset_predict_df.fillna(0)

dataset_predict_df = pd.merge(dataset_predict_df, item_df, on=['item_id'], how='left')
dataset_predict_df = pd.merge(dataset_predict_df, shop_df, on=['shop_id'], how='left')
dataset_predict_df = pd.merge(dataset_predict_df, category_df, on=['item_category_id'], how='left')

dataset_predict_df['month'] = dataset_predict_df['date_block_num'].apply(lambda x: (x % 12) + 1)

sc_df = _agg(dataset_predict_df, ['shop_id', 'item_category_id'], 'sc')
i_df = _agg(dataset_predict_df, ['item_id'], 'i')
it_df = _agg(dataset_predict_df, ['item_category_id'], 'it')
s_df = _agg(dataset_predict_df, ['shop_id'], 's')

dataset_predict_df = pd.merge(dataset_predict_df, sc_df, on=['shop_id', 'item_category_id'], how='left')
dataset_predict_df = pd.merge(dataset_predict_df, i_df, on=['item_id'], how='left')
dataset_predict_df = pd.merge(dataset_predict_df, it_df, on=['item_category_id'], how='left')
dataset_predict_df = pd.merge(dataset_predict_df, s_df, on=['shop_id'], how='left')

dataset_predict_df = dataset_predict_df.drop(['city', 'type', 'subtype', 'item_name', 'shop_name', 'item_category_name'], axis=1)
dataset_predict_df = _transfer_type(dataset_predict_df, ['item_category_id', 'city_code', 'type_code', 'subtype_code', 'month'], np.int16)
dataset_predict_df.head(20)

/Users/leewind/.local/share/virtualenvs/leewind-p6XO93Th/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,shop_id,item_id,r1,r2,r3,r4,r5,r6,r7,r8,...,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12
0,5,5037,0.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,...,0.23877,0.171021,0.167847,0.175293,0.174316,0.172729,0.164062,0.210449,0.312256,0.203979
1,5,5320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.23877,0.171021,0.167847,0.175293,0.174316,0.172729,0.164062,0.210449,0.312256,0.203979
2,5,5233,1.0,3.0,1.0,0.0,2.0,3.0,0.0,0.0,...,0.23877,0.171021,0.167847,0.175293,0.174316,0.172729,0.164062,0.210449,0.312256,0.203979
3,5,5232,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.23877,0.171021,0.167847,0.175293,0.174316,0.172729,0.164062,0.210449,0.312256,0.203979
4,5,5268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.23877,0.171021,0.167847,0.175293,0.174316,0.172729,0.164062,0.210449,0.312256,0.203979
5,5,5039,1.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.23877,0.171021,0.167847,0.175293,0.174316,0.172729,0.164062,0.210449,0.312256,0.203979
6,5,5041,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.23877,0.171021,0.167847,0.175293,0.174316,0.172729,0.164062,0.210449,0.312256,0.203979
7,5,5046,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.23877,0.171021,0.167847,0.175293,0.174316,0.172729,0.164062,0.210449,0.312256,0.203979
8,5,5319,0.0,3.0,2.0,3.0,4.0,2.0,2.0,3.0,...,0.23877,0.171021,0.167847,0.175293,0.174316,0.172729,0.164062,0.210449,0.312256,0.203979
9,5,5003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.23877,0.171021,0.167847,0.175293,0.174316,0.172729,0.164062,0.210449,0.312256,0.203979


In [68]:
predict_dmatrix = xgb.DMatrix(dataset_predict_df[features])
pred = model.predict(predict_dmatrix)

In [69]:
submission = pd.DataFrame({
    "ID": test_df.index, 
    "item_cnt_month": pred.clip(0, 20)
})

submission.to_csv('submission.csv', index=False)

# Reference

## Normal ML Analysis

+ [Feature engineering, xgboost](https://www.kaggle.com/dlarionov/feature-engineering-xgboost)
+ [Predicting sales using Lightgbm](https://www.kaggle.com/sanket30/predicting-sales-using-lightgbm)

## Time Series

对于revenge的预测很有帮助，但是对于单个店的单个商品而言没有实际的意义，这种预测方式的学习非常有益处

+ [AR(I)MA时间序列建模过程——步骤和python代码](https://www.jianshu.com/p/cced6617b423)
+ [python时间序列分析](http://www.cnblogs.com/foley/p/5582358.html)
+ [AR、MA及ARMA模型](https://zhuanlan.zhihu.com/p/22248464)
+ [Time Series with Python (ODSC) STA.ipynb](https://github.com/ultimatist/ODSC17/blob/master/Time%20Series%20with%20Python%20(ODSC)%20STA.ipynb)
+ [Getting Started with Time Series](https://pyflux.readthedocs.io/en/latest/getting_started.html)
+ [Welcome to Statsmodels’s Documentation](http://www.statsmodels.org/devel/index.html)